# Recursive Character Text Splitter
`RecursiveCharacterTextSplitter`는  **가장 지능적인** 텍스트 분할 도구입니다.  
CharacterTextSplitter의 단일 separator 방식과 달리, **여러 separator를 순차적으로 시도**하여 가장 자연스러운 분할점을 찾습니다.  
대부분의 RAG 시스템에서 권장되는 방식입니다
### 주요 특징
1. **지능적 분할**: 여러 separator를 우선순위에 따라 순차 적용합니다
2. **자연스러운 경계**: 문단 → 문장 → 단어 순으로 자연스러운 분할을 수행합니다
3. **유연한 설정**: separator 목록을 사용자가 정의 가능합니다


In [ ]:
from langchain_community.document_loaders import TextLoader

# test.txt 파일의 경로 설정
file_path = "../data/ai.txt"

# TextLoader를 사용하여 텍스트 파일 로드
loader = TextLoader(file_path, encoding='utf-8')

# 문서 로드
document = loader.load()

print(f"로드된 문서 수: {len(document)}")
print(document)

## 2. CharacterTextSplitter와의 비교

`RecursiveCharacterTextSplitter`의 핵심 차이점을 이해해보겠습니다.

### 주요 차이점

| 구분 | CharacterTextSplitter | RecursiveCharacterTextSplitter |
|------|----------------------|-------------------------------|
| **Separator** | 단일 separator | 여러 separator 목록 (우선순위) |
| **분할 방식** | 고정된 기준으로 분할 | 적응적으로 최적 기준 선택 |
| **자연스러움** | 기계적 분할 | 의미 단위 보존하며 분할 |
| **사용 복잡도** | 간단 | 약간 복잡하지만 더 지능적 |
| **결과 품질** | 기본적 | 더 자연스럽고 의미 있는 분할 |

### Recursive 동작 원리

1. **1단계**: 첫 번째 separator(`"\n\n"`)로 분할 시도
2. **2단계**: 청크가 너무 크면 두 번째 separator(`"\n"`)로 재분할
3. **3단계**: 여전히 크면 세 번째 separator(`". "`)로 재분할
4. **4단계**: 마지막으로 네 번째 separator(`" "`)로 단어 단위 분할
5. **최종**: 모든 separator를 시도해도 크면 강제로 문자 단위 분할


**실습** - 눈으로 확인하기위해 chunck_size를 낮게 설정.

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

character_splitter = CharacterTextSplitter(
    separator="\n",       # 문단 구분만 사용
    chunk_size=100,         # 200자 제한
    chunk_overlap=20
)

character_chunks = character_splitter.split_documents(document)

print(f"총 {len(character_chunks)}개 청크 생성\n")

for i, chunk in enumerate(character_chunks, 1):
    chunk_length = len(chunk.page_content)
    print(f"청크 {i} ({chunk_length}자):")
    print(f"'{chunk.page_content}'")


단일 문단으로 split을 했을때 청크가 너무 길어지거나 너무 많은 문장이 하나의 청크에 섞이는 문제가 발생한다.  
이럴때 `RecursiveCharacterTextSplitter`를 사용할 수 있다.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

recursive_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " "],  # 문단 → 줄 → 문장 → 단어 순
    chunk_size=100,    
    chunk_overlap=20
)

recursive_chunks = recursive_splitter.split_documents(document)

print(f"총 {len(recursive_chunks)}개 청크 생성\n")

for i, chunk in enumerate(recursive_chunks, 1):
    chunk_length = len(chunk.page_content)
    print(f"청크 {i} ({chunk_length}자):")
    print(f"'{chunk.page_content}'")